In [ ]:
""" import settings """
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_ssep_global import *

In [ ]:
""" load data, define static params """
# load bad channel
bad_ch_idx_dir = f'{DATA_DIR}/1_bad_channels'
bad_chs = np.load(f"{bad_ch_idx_dir}/bad_ch_idx.npy")

# load downsampled data
fs, Ts = FS/RS, TS*RS
ds_data_dir = f'{DATA_DIR}/5_downsample'
t = np.load(f"{ds_data_dir}/t_{RS}.npy")

# SSEP index
sep_idxs = np.where(np.logical_and(t > SEP_T0, t < SEP_T1))[0]

# Baseline index
baseline_idxs = np.where(np.logical_and(t > BASELINE_T0, t < BASELINE_T1))[0]

# define "z-score window"
zscore_t0, zscore_t1 = BASELINE_T0, SEP_T1
zscore_idxs = np.where(np.logical_and(t > zscore_t0, t < zscore_t1))[0]

Z-score Time Domain Data

In [ ]:
""" define path for saving z-scored data """
save_dir = f'{DATA_DIR}/7_zscore'
if not os.path.exists(save_dir):
   os.makedirs(save_dir)

In [ ]:
""" z-score time domain data. normalize against window that covers baseline and SSEP """
for idx, stim_label in enumerate(STIM_LABELS):
    fn_label = stim_label.replace(" ", "_").lower()
    ds_segs = np.load(f"{ds_data_dir}/{fn_label}_ds_{RS}_segs.npy")

    num_trials = ds_segs.shape[1]
    
    zscore_segs = np.full(ds_segs.shape, np.nan)

    # ch_data: trial*time
    for ch, ch_data in enumerate(ds_segs):
        # use all-trial sigma
        sigma = np.nanstd(ch_data[:,zscore_idxs])
        for trial, trial_data in enumerate(ch_data):
            if np.isnan(trial_data[0]):
                continue

            # because baseline fluctuates, use per-trial mu
            mu = np.mean(trial_data[baseline_idxs])
            zscore_segs[ch][trial] = (trial_data - mu)/sigma
    
    np.save(f'{save_dir}/{fn_label}_ds_{RS}_zscore.npy', zscore_segs)